> ### Lab-01 `Inference ` : `Without Prompt` vs `Zero Shot` vs `One Shot` vs `Few Shot`

In [3]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
      torch==1.13.1 \
      torchdata==0.5.1 --quiet

%pip install \
      transformers==4.27.2 \
      datasets==2.11.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 66.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torchdata==0.6.1, but you have torchdata 0.5.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.


In [5]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig


 #### Case1 : Summarize Dialogue without prompt Engineering

In [6]:
hg_dataset_name = 'knkarthick/dialogsum'
dataset = load_dataset(hg_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
eg_indices = [50,150]
for i, index in enumerate(eg_indices):
  print(f'--- --- '*10)
  print(f"\n Example at Index : {index}")
  print(f"\n Dialogue : \n\n{dataset['test'][index]['dialogue']}")
  print(f"\n Summary : \n {dataset['test'][index]['summary']}")

--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Example at Index : 50

 Dialogue : 

#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don't think this is fun, they're not meant to be our friends.
#Person2#: You mean your friends. I think it's cruel.
#Person1#: Yeah. But it's fun. Look at those two ugly old ladies. . . or are they men?
#Person2#: Hurry! Get a shot!. . . Hand it over!
#Person1#: I knew you'd come around. . .

 Summary : 
 #Person1# is about to make a prank. #Person2# thinks it's cruel at first but then joins.
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Example at Index : 150

 Dialogue : 

#Person1#: Taxi!
#Person2#: Where will you go, sir?
#Person1#: Friendship Hotel.
#Person2#: OK, it's not far from here.
#Person1#: I have something important to do, can you fast the speed?
#Person2#: Sure, I'll try my best. Here

In [32]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

##### How does Tokenizing and De-tokenizing Works ?

In [37]:
sentence = 'Where do you work?, Harry'

print(f'\nSENTENCE: \n{sentence}')

sentence_encoded = tokenizer(sentence,return_tensors='pt')

print(f'\nENCODED SENTENCE: \n{sentence_encoded}')

sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0],
                                    skip_special_tokens=True)
print(f'\nDECODED SENTENCE: \n{sentence_decoded}')




SENTENCE: 
Where do you work?, Harry

ENCODED SENTENCE: 
{'input_ids': tensor([[2840,  103,   25,  161,   58,    6, 8929,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

DECODED SENTENCE: 
Where do you work?, Harry


##### Model Output withput Prompts

In [53]:
for i, index in enumerate(eg_indices):
  dialogue = dataset['test'][index]['dialogue']
  sentence_encoded = tokenizer(dialogue,return_tensors='pt')
  model_output_tokens = model.generate(inputs=sentence_encoded['input_ids'],max_new_tokens=50,)
  model_output_decoded = tokenizer.decode(model_output_tokens[0],
                                    skip_special_tokens=True)

  print(f'--- --- '*10)
  print(f"\n Example at Index : {index}")
  print(f"\n Dialogue : \n\n{dialogue}")
  print(f"\n Summary : \n {dataset['test'][index]['summary']}")
  print(f"\n model_output_decoded : \n {model_output_decoded}")

--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Example at Index : 50

 Dialogue : 

#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don't think this is fun, they're not meant to be our friends.
#Person2#: You mean your friends. I think it's cruel.
#Person1#: Yeah. But it's fun. Look at those two ugly old ladies. . . or are they men?
#Person2#: Hurry! Get a shot!. . . Hand it over!
#Person1#: I knew you'd come around. . .

 Summary : 
 #Person1# is about to make a prank. #Person2# thinks it's cruel at first but then joins.

 model_output_decoded : 
 #Person1#: Yeah.
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Example at Index : 150

 Dialogue : 

#Person1#: Taxi!
#Person2#: Where will you go, sir?
#Person1#: Friendship Hotel.
#Person2#: OK, it's not far from here.
#Person1#: I have something important to do, can you fast the sp

#### Case 2 : Zero Shot Inference with Instruction Prompt

In [54]:
for i, index in enumerate(eg_indices):

  dialogue = dataset['test'][index]['dialogue']

  prompt = f"""
  Dialogue:
  {dialogue}

  Please Summarize the above dialogue
  """

  sentence_encoded = tokenizer(prompt,return_tensors='pt')
  model_output_tokens = model.generate(inputs=sentence_encoded['input_ids'],max_new_tokens=50,)
  model_output_decoded = tokenizer.decode(model_output_tokens[0],
                                    skip_special_tokens=True)

  print(f'--- --- '*10)
  print(f"\n Example at Index : {index}")
  print(f"\n Dialogue : \n\n{dialogue}")
  print(f"\n Summary : \n {dataset['test'][index]['summary']}")
  print(f"\n model_output_decoded : \n {model_output_decoded}")

--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Example at Index : 50

 Dialogue : 

#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don't think this is fun, they're not meant to be our friends.
#Person2#: You mean your friends. I think it's cruel.
#Person1#: Yeah. But it's fun. Look at those two ugly old ladies. . . or are they men?
#Person2#: Hurry! Get a shot!. . . Hand it over!
#Person1#: I knew you'd come around. . .

 Summary : 
 #Person1# is about to make a prank. #Person2# thinks it's cruel at first but then joins.

 model_output_decoded : 
 Person1 wants to make a few enemies.
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Example at Index : 150

 Dialogue : 

#Person1#: Taxi!
#Person2#: Where will you go, sir?
#Person1#: Friendship Hotel.
#Person2#: OK, it's not far from here.
#Person1#: I have something important to do,

#### Case 3 : One Shot Inference with Instruction Prompt

In [55]:
def one_shot_inference(example_index):

  ex_dialogue = dataset['test'][example_index]['dialogue']
  ex_summary = dataset['test'][example_index]['summary']

  print(f"\n ex_Dialogue : \n {ex_dialogue}")
  print(f"\n ex_Summary : \n {ex_summary}")

  for i, index in enumerate(eg_indices):

    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Dialogue:
    {ex_dialogue}

    Please Summarize the above dialogue
    {ex_summary}

    Dialogue:
    {dialogue}

    Please Summarize the above dialogue
    """


    sentence_encoded = tokenizer(prompt,return_tensors='pt')
    model_output_tokens = model.generate(inputs=sentence_encoded['input_ids'],max_new_tokens=50,)
    model_output_decoded = tokenizer.decode(model_output_tokens[0],
                                      skip_special_tokens=True)

    print(f'--- --- '*10)
    print(f"\n Example at Index : {index}")
    print(f"\n Dialogue : \n\n{dialogue}")
    print(f"\n Summary : \n {dataset['test'][index]['summary']}")
    print(f"\n model_output_decoded : \n {model_output_decoded}")

In [57]:
one_shot_inference(30)


 ex_Dialogue : 
 #Person1#: Where are you going for your trip?
#Person2#: I think Hebei is a good place.
#Person1#: But I heard the north of China are experiencing severe sandstorms!
#Person2#: Really?
#Person1#: Yes, it's said that Hebes was experiencing six degree strong winds.
#Person2#: How do these storms affect the people who live in these areas?
#Person1#: The report said the number of people with respiratory tract infections tended to rise after sandstorms. The sand gets into people's noses and throats and creates irritation.
#Person2#: It sounds that sandstorms are trouble for everybody!
#Person1#: You are quite right.

 ex_Summary : 
 #Person2# plans to have a trip in Hebei but #Person1# says there are sandstorms in there.
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Example at Index : 50

 Dialogue : 

#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they d

#### Case 4: Few Shot Inference

In [76]:
def few_shot_inference(example_indices):
  prompt = ""
  for example_index in example_indices:
    ex_dialogue = dataset['test'][example_index]['dialogue']
    ex_summary = dataset['test'][example_index]['summary']

    prompt += f"""
Dialogue:
{ex_dialogue}

Please Summarize the above dialogue
{ex_summary}
    """

  print(f"\nFewshot Examples :\n{prompt}")

  for i, index in enumerate(eg_indices):

    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt += f"""
    Dialogue:
    {dialogue}

    Please Summarize the above dialogue
    """


    sentence_encoded = tokenizer(prompt,return_tensors='pt')
    model_output_tokens = model.generate(inputs=sentence_encoded['input_ids'],max_new_tokens=50,)
    model_output_decoded = tokenizer.decode(model_output_tokens[0],
                                      skip_special_tokens=True)

    print(f'--- --- '*10)
    print(f"\n Example at Index : {index}")
    print(f"\n Dialogue : \n\n{dialogue}")
    print(f"\n Summary : \n {dataset['test'][index]['summary']}")
    print(f"\n model_output_decoded : \n {model_output_decoded}")

In [77]:
example_indices = [11,24,35,70]
few_shot_inference(example_indices)


 Fewshot Examples :

Dialogue:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

Please Summarize the above dialogue
#Person1# has a dance with Brian at Brian's birthday party. Brian thinks #Person1# looks great and is popular.
    
Dialogue:
#Person1#: Steven, I need badly your help.
#Person2#: What's the matter?
#Person1#: My wife has found that I have an affair with my secre

#### Case 5: Inference by changing Config Setting

> Temparature = 0.1 --> Output is more Consrvative and will give the same response again and again

>Temparature = 2 --> Output is more Randomized and wil give different output

##### One-shot-Inference by changing Config Setting

In [99]:
def one_shot_config_inference(example_index, config_settings):

  ex_dialogue = dataset['test'][example_index]['dialogue']
  ex_summary = dataset['test'][example_index]['summary']

  # print(f"\n ex_Dialogue : \n {ex_dialogue}")
  # print(f"\n ex_Summary : \n {ex_summary}")

  for i, index in enumerate(eg_indices):

    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Dialogue:
    {ex_dialogue}

    Please Summarize the above dialogue
    {ex_summary}

    Dialogue:
    {dialogue}

    Please Summarize the above dialogue
    """


    sentence_encoded = tokenizer(prompt,return_tensors='pt')
    model_output_tokens = model.generate(inputs=sentence_encoded['input_ids'],
                                         generation_config = config_settings)
    model_output_decoded = tokenizer.decode(model_output_tokens[0],
                                      skip_special_tokens=True)

    print(f'--- --- '*10)
    # print(f"\n Example at Index : {index}")
    # print(f"\n Dialogue : \n\n{dialogue}")
    print(f"\n Summary : \n {dataset['test'][index]['summary']}")
    print(f"\n model_output_decoded : \n {model_output_decoded}")

In [101]:
example_index = 20
config_settings = GenerationConfig(max_new_tokens=50,temparature=2,do_sample=True)
one_shot_config_inference(example_index, config_settings)

--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Summary : 
 #Person1# is about to make a prank. #Person2# thinks it's cruel at first but then joins.

 model_output_decoded : 
 People will make enemies if they do not agree with Person1 despite the fact that they like to help others.
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Summary : 
 #Person1# takes a taxi to the Friendship Hotel for something important.

 model_output_decoded : 
 The taxi meter reads 15 yuan, but he will pay the driver 20 yuan.


##### Few-shot-Inference by changing Config Setting

In [95]:
def few_shot_config_inference(example_indices, config_settings):
  prompt = ""
  for example_index in example_indices:
    ex_dialogue = dataset['test'][example_index]['dialogue']
    ex_summary = dataset['test'][example_index]['summary']

    prompt += f"""
Dialogue:
{ex_dialogue}

Please Summarize the above dialogue
{ex_summary}
    """

  # print(f"\nFewshot Examples :\n{prompt}")

  for i, index in enumerate(eg_indices):

    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt += f"""
    Dialogue:
    {dialogue}

    Please Summarize the above dialogue
    """


    sentence_encoded = tokenizer(prompt,return_tensors='pt')
    model_output_tokens = model.generate(inputs=sentence_encoded['input_ids'],max_new_tokens=50,)
    model_output_decoded = tokenizer.decode(model_output_tokens[0],
                                      skip_special_tokens=True)

    print(f'--- --- '*10)
    # print(f"\n Example at Index : {index}")
    # print(f"\n Dialogue : \n\n{dialogue}")
    print(f"\n Summary : \n {dataset['test'][index]['summary']}")
    print(f"\n model_output_decoded : \n {model_output_decoded}")

In [98]:
example_indices = [11,24,35]
config_settings = GenerationConfig(max_new_tokens=50,temparature=1,do_sample=True)
few_shot_config_inference(example_indices,config_settings)

--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Summary : 
 #Person1# is about to make a prank. #Person2# thinks it's cruel at first but then joins.

 model_output_decoded : 
 The two ugly old ladies are not meant to be friends.
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

 Summary : 
 #Person1# takes a taxi to the Friendship Hotel for something important.

 model_output_decoded : 
 The taxi driver will pick up Person1 at Friendship Hotel at 20 yuan.
